In [1]:
!git clone https://github.com/mohamed-stifi/darija-llm-lab.git

fatal: destination path 'darija-llm-lab' already exists and is not an empty directory.


In [2]:
import os
os.chdir("darija-llm-lab")

In [3]:
%%capture
!python -m pip install --upgrade pip
!pip install uv
!uv pip install -r requirements.txt

In [4]:
#import os
#os.chdir("../")
!pwd

/kaggle/working/darija-llm-lab


In [5]:
from darija_llm_lab.components.data_ingestion.data_ingestion import DataIngestion
from darija_llm_lab.utils.common import ConfigurationManager

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-08-15 21:27:58.916160: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755293279.114654     116 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755293279.172705     116 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


## Get configs

In [6]:
config_manager = ConfigurationManager(config_filepath="config/config.yaml")

In [7]:
# 1. Get configs
model_cfg = config_manager.get_model_config()
dapt_cfg = config_manager.get_dapt_config()
sft_config = config_manager.get_sft_config()
new_tokens = config_manager.new_tokens
wandb_config = config_manager.get_wandb_config()
optuna_config = config_manager.get_optuna_config()

In [8]:
sft_config

SFTConfig(input_model_path=PosixPath('artifacts/models/gemma-3n-E2B-it-darija-adapted'), output_adapters_path=PosixPath('artifacts/sft_adapters'), data=SFTDataConfig(dataset_id='MBZUAI-Paris/Darija-SFT-Mixture', test_size=0.1, seed=1234, debug_mode=True, debug_split_size=3000), peft=SFTPEFTConfig(r=16, lora_alpha=16, lora_dropout=0.05, bias='none', random_state=3407, target_modules=BoxList(['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'])), trainer=SFTTrainerConfig(output_dir='artifacts/sft_training_results', per_device_train_batch_size=10, gradient_accumulation_steps=10, eval_strategy='steps', eval_steps=10, save_strategy='steps', save_steps=30, warmup_steps=10, max_steps=100, learning_rate=2e-05, logging_steps=5, seed=3407, metric_for_best_model='eval_loss', load_best_model_at_end=True, greater_is_better=False, early_stopping_patience=3, optim='adamw_torch', weight_decay=0.01))

In [9]:
dapt_cfg

DAPTConfig(output_model_path=PosixPath('artifacts/models/gemma-3n-E2B-it-darija-adapted'), data=DAPTDataConfig(dataset_id='mohamed-stifi/darija-combined-dataset', text_column='text', debug_mode=True, debug_split_size=5000), trainer=DAPTTrainerConfig(output_dir='artifacts/dapt_training_results', per_device_train_batch_size=8, gradient_accumulation_steps=8, max_steps=200, learning_rate=5e-06, logging_steps=10, save_steps=50, warmup_steps=20, seed=3407, optim='adamw_torch', weight_decay=0.01))

In [10]:
dapt_cfg.data

DAPTDataConfig(dataset_id='mohamed-stifi/darija-combined-dataset', text_column='text', debug_mode=True, debug_split_size=5000)

# DAPT Model.

## Load and prepare model

In [11]:
new_tokens

['علاش', 'ماشي', 'واش']

In [12]:
from darija_llm_lab.components.model_loader import ModelLoader

# 2. Load and prepare model
model_loader = ModelLoader(model_cfg, new_tokens)
model, tokenizer = model_loader.load_model_and_tokenizer()

--- Loading model from: unsloth/gemma-3n-E2B-it ---
==((====))==  Unsloth 2025.7.2: Fast Gemma3N patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3N does not support SDPA - switching to eager!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/469M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

## perform embedding surgery

In [13]:
model, tokenizer = model_loader.perform_embedding_surgery()


--- Step 2: Performing Embedding Surgery ---
Adding 3 new tokens. New vocabulary size will be: 262403
Model embedding matrices resized successfully.
Updating model configuration...
Updating tokenizer...

--- Embedding Surgery Complete ---


In [14]:
model.config.text_config.vocab_size

262411

In [15]:
model.get_input_embeddings()

Gemma3nTextScaledWordEmbedding(262408, 2048, padding_idx=0)

In [16]:
len(tokenizer.tokenizer.vocab)

262403

## prepare model for dapt

In [17]:
model = model_loader.prepare_model_for_dapt()


--- Preparing model for DAPT (training embeddings and lm_head) ---


/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:693: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


trainable params: 2,115,648 || all params: 5,441,560,064 || trainable%: 0.0389
trainable params: 539,516,992 || all params: 5,441,560,064 || trainable%: 9.9147

Trainable parameters:
base_model.model.model.language_model.embed_tokens.weight
base_model.model.lm_head.lora_A.default.weight
base_model.model.lm_head.lora_B.default.weight


In [18]:
model.language_model.embed_tokens.weight.requires_grad

True

In [19]:
model.get_input_embeddings().weight.requires_grad

True

# SFT Model.

In [22]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
10.988 GB of memory reserved.


In [23]:
from darija_llm_lab.components.model_loader import ModelLoader

# 2. Load and prepare model
model_loader = ModelLoader(model_cfg, new_tokens)
model, tokenizer = model_loader.load_model_and_tokenizer()

--- Loading model from: unsloth/gemma-3n-E2B-it ---
==((====))==  Unsloth 2025.7.2: Fast Gemma3N patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3N does not support SDPA - switching to eager!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/accelerate/utils/modeling.py:1614: UserWarning: The following device_map keys do not match any submodules in the model: ['model.audio_tower.conformer.4.attention.attn.per_dim_scale']
  warnings.warn(


In [24]:
model = model_loader.apply_sft_peft(sft_config.peft)

--- Applying PEFT (LoRA) for SFT ---


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Making `model.base_model.model.model.language_model` require gradients
trainable params: 21135360 || all params: 4528651712 || trainable%: 0.4667
